In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
os.chdir('../../data/face recognition')

In [3]:
def read_images(path, image_size):
    names=[]
    training_images, training_labels = [], []
    label=0
    for dirname, subdirnames, filenames in os.walk(path):
        for subdirname in subdirnames:
            names.append(subdirname)
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                img = cv.imread(os.path.join(subject_path, filename), cv.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                img = cv.resize(img, image_size)
                training_images.append(img)
                training_labels.append(label)
            label+=1
    training_images = np.asarray(training_images, np.uint8)
    training_labels = np.asarray(training_labels, np.int32)
    return names, training_images, training_labels

In [4]:
path = 'dataset'
size = (200, 200)
names, training_images, training_labels = read_images(path, size)

In [6]:
face_cascade = cv.CascadeClassifier('../cascades/haarcascade_frontalface_default.xml')

### Performing face recognition with Eigenfaces

In [5]:
model = cv.face.EigenFaceRecognizer_create()
model.train(training_images, training_labels)

In [ ]:
cap = cv.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        for x, y, w, h in faces:
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            roi_gray = gray[x:x+w, y:y+h]
            if roi_gray.size == 0: continue
            roi_gray = cv.resize(roi_gray, size)
            label, confidence = model.predict(roi_gray)
            # text = '%s, confidence=%.2f' % (names[label], confidence)
            text1 = names[label]
            text2 = 'confidence=%.2f' % confidence
            cv.putText(frame, text1, (x, y-25), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            cv.putText(frame, text2, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 0), 2)
        cv.imshow('face recognition', frame)
        if cv.waitKey(30) == 27:
            break
    else:
        break
cap.release()
cv.destroyAllWindows()

In [8]:
model.save('amn.h5')

### Performing face recognition with Fisherfaces

In [7]:
Fisherfaces = cv.face.FisherFaceRecognizer_create()
Fisherfaces.train(training_images, training_labels)

In [ ]:
cap = cv.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        for x, y, w, h in faces:
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            roi_gray = gray[x:x+w, y:y+h]
            if roi_gray.size == 0: continue
            roi_gray = cv.resize(roi_gray, size)
            label, confidence = Fisherfaces.predict(roi_gray)
            name = names[label]
            confidenc_e = 'confidence=%.2f' % confidence
            cv.putText(frame, name, (x, y-25), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            cv.putText(frame, confidenc_e, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 0), 2)
        cv.imshow('face recognition', frame)
        if cv.waitKey(30) == 27:
            break
    else:
        break
cap.release()
cv.destroyAllWindows()